In [1]:
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import os
from functools import reduce
import warnings 
warnings.filterwarnings('ignore')

In [2]:
# 분류모델 import 
# Logistic Regression,SGD,KNN,SVM,Decision Tree, Random Forest, LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report,roc_auc_score,f1_score

lr = LogisticRegression()
knn = KNeighborsClassifier()
svm = SVC()
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
lgbm = LGBMClassifier()

In [77]:
df_coldwave = pd.read_csv('./data/preprocessed/df_coldwave.csv',encoding='cp949',index_col=0)
df_kr = pd.read_csv('./data/preprocessed/국내기후(백령도제외).csv',index_col=0)
df_near = pd.read_csv('./data/preprocessed/preprocessed_주변국 기상데이터_199601to202108.csv',index_col=0)

In [58]:
df_near.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271875 entries, 0 to 9374
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   도시명     271875 non-null  object 
 1   일시      271875 non-null  object 
 2   평균기온    271875 non-null  float64
 3   최고기온    271875 non-null  float64
 4   최저기온    271875 non-null  float64
 5   습도      271875 non-null  float64
 6   풍향      271875 non-null  float64
 7   풍속      271875 non-null  float64
 8   강수량     271875 non-null  float64
 9   해면기압    271875 non-null  float64
 10  이슬점 온도  271875 non-null  float64
 11  군집      271875 non-null  object 
dtypes: float64(9), object(3)
memory usage: 27.0+ MB


In [78]:
# 도시명 집계함수 사ㅏ용 불가능 -> 오브젝트 제거 
# 그룹하는데 기준할땐 사용할수있음 
df_near = df_near.drop('도시명',axis=1).groupby(['군집','일시']).mean().reset_index()
# 군집과 일시로 묶음 
# 군집별일시

In [64]:
df_near.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65625 entries, 0 to 65624
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   군집      65625 non-null  object 
 1   일시      65625 non-null  object 
 2   평균기온    65625 non-null  float64
 3   최고기온    65625 non-null  float64
 4   최저기온    65625 non-null  float64
 5   습도      65625 non-null  float64
 6   풍향      65625 non-null  float64
 7   풍속      65625 non-null  float64
 8   강수량     65625 non-null  float64
 9   해면기압    65625 non-null  float64
 10  이슬점 온도  65625 non-null  float64
dtypes: float64(9), object(2)
memory usage: 5.5+ MB


In [65]:
df_near

,군집,일시,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도
0,n1,1996-01-01,-3.639583,0.500000,-8.087500,48.419643,218.750000,1.169643,0.000000,1022.948810,-13.566369
1,n1,1996-01-02,0.900000,3.750000,-2.812500,71.203125,165.000000,2.515625,0.446631,1017.043750,-3.859375
2,n1,1996-01-03,-6.095313,0.075000,-8.775000,52.091518,253.750000,4.433265,0.446631,1021.464062,-14.331250
3,n1,1996-01-04,-7.325000,-1.975000,-12.437500,59.312500,198.348983,1.484375,0.445600,1025.557812,-14.362500
4,n1,1996-01-05,-1.073214,1.175000,-3.050000,80.558036,230.524917,1.966518,1.623677,1021.357143,-4.158482
...,...,...,...,...,...,...,...,...,...,...,...
65620,n7,2021-08-27,25.229167,28.333333,22.500000,78.830556,240.000000,2.125000,-0.047822,1012.870833,21.366667
65621,n7,2021-08-28,26.490476,30.100000,23.133333,81.301190,106.666667,1.404762,-0.739609,1012.662500,22.993452
65622,n7,2021-08-29,25.787500,29.033333,23.233333,87.795833,210.000000,2.083333,-0.702848,1013.058333,23.450000
65623,n7,2021-08-30,27.412500,30.700000,25.233333,83.704167,220.000000,1.916667,-0.625120,1012.554167,24.179167


In [86]:
df_coldwave.drop_duplicates(inplace=True)

In [87]:
df_coldwave

,연도,날짜,지점,일시,재난일어난날
0,1973,103,이천,1973-01-03,1973-01-03
1,1973,103,수원,1973-01-03,1973-01-03
2,1973,103,임실,1973-01-03,1973-01-03
3,1973,103,서산,1973-01-03,1973-01-03
4,1973,103,대관령,1973-01-03,1973-01-03
...,...,...,...,...,...
21982,2022,224,장수,2022-02-24,2022-02-24
21983,2022,224,대관령,2022-02-24,2022-02-24
21984,2022,224,인제,2022-02-24,2022-02-24
21985,2022,224,이천,2022-02-24,2022-02-24


In [43]:
from functools import reduce
import pandas as pd
dfs = [df_coldwave, df_kr, df_near]
cold_kr_near = reduce(lambda left, right: pd.merge(left,right,left_index=True, right_index=True), dfs)

In [45]:
cold_kr_near.drop_duplicates()

,재난일어난날,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,...,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
일시,,,,,,,,,,,,,,,,,,,,,
1996-01-01,1996-01-01,서울,-3.2,-7.9,2.5,0.0,0.0,6.0,340.0,3.5,...,-5.528571,-0.1,-9.9,53.571429,330.0,0.571429,0.0,1018.685714,-13.657143,n2
1996-01-01,1996-01-01,서울,-3.2,-7.9,2.5,0.0,0.0,6.0,340.0,3.5,...,-17.157143,-8.0,-24.0,70.285714,50.0,0.571429,0.0,1021.942857,-21.457143,n2
1996-01-01,1996-01-01,서울,-3.2,-7.9,2.5,0.0,0.0,6.0,340.0,3.5,...,-15.250000,-7.5,-20.9,74.166667,0.0,0.000000,0.0,1026.083333,-18.933333,n2
1996-01-01,1996-01-01,서울,-3.2,-7.9,2.5,0.0,0.0,6.0,340.0,3.5,...,-6.757143,-2.5,-10.9,63.000000,340.0,0.428571,0.0,1023.066667,-13.042857,n1
1996-01-01,1996-01-01,서울,-3.2,-7.9,2.5,0.0,0.0,6.0,340.0,3.5,...,-4.600000,1.2,-11.2,43.857143,330.0,0.857143,0.0,1018.585714,-15.657143,n2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-03,2021-03-03,제주,9.2,4.8,12.8,0.0,0.0,11.7,50.0,6.1,...,6.866667,11.1,4.4,52.454167,10.0,5.500000,0.0,1029.450000,-2.487500,n5
2021-03-03,2021-03-03,제주,9.2,4.8,12.8,0.0,0.0,11.7,50.0,6.1,...,10.175000,14.1,7.4,36.837500,360.0,4.125000,0.0,1019.612500,-4.337500,n7
2021-03-03,2021-03-03,제주,9.2,4.8,12.8,0.0,0.0,11.7,50.0,6.1,...,2.987500,4.3,-1.3,68.987500,40.0,4.625000,0.0,1014.800000,-2.162500,n6


In [27]:
df_coldwave.drop(['연도','날짜','지점'],axis=1,inplace=True)

In [175]:
df_coldwave

,일시,재난일어난날
0,1973-01-03,1973-01-03
1,1973-01-03,1973-01-03
2,1973-01-03,1973-01-03
3,1973-01-03,1973-01-03
4,1973-01-03,1973-01-03
...,...,...
21982,2022-02-24,2022-02-24
21983,2022-02-24,2022-02-24
21984,2022-02-24,2022-02-24
21985,2022-02-24,2022-02-24


In [176]:
# 중복된 날짜 제거 
df_coldwave = df_coldwave.drop_duplicates()

In [177]:
df_coldwave

,일시,재난일어난날
0,1973-01-03,1973-01-03
16,1973-01-04,1973-01-04
18,1973-01-06,1973-01-06
19,1973-01-11,1973-01-11
22,1973-01-12,1973-01-12
...,...,...
21950,2022-02-20,2022-02-20
21952,2022-02-21,2022-02-21
21960,2022-02-22,2022-02-22
21963,2022-02-23,2022-02-23


In [6]:
df_kr.rename(columns={'시간':'일시'},inplace=True)

In [179]:
df_kr

,일시,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,...,합계 일사,일 최심신적설,일 최심적설,합계 3시간 신적설,평균 전운량,평균 중하층운량,평균 지면온도,합계 대형증발량,합계 소형증발량,안개 계속 시간
0,1990-01-01,서울,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,5.0,...,4.58,0.0,0.0,0.0,3.5,3.6,-1.5,2.721902,0.9,0.0
1,1990-01-02,서울,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,4.8,...,3.44,0.1,0.1,0.1,3.5,3.8,-2.9,2.721902,0.8,0.0
2,1990-01-03,서울,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,5.0,...,7.63,0.0,0.0,0.0,0.0,0.0,-4.4,2.721902,1.4,0.0
3,1990-01-04,서울,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,3.0,...,5.84,0.0,0.0,0.0,0.5,1.8,-3.6,2.721902,1.0,0.0
4,1990-01-05,서울,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,7.2,...,0.36,0.6,0.6,0.6,9.0,3.6,-0.6,2.721902,1.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11683,2021-12-27,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,6.8,...,1.37,0.5,1.2,0.5,8.1,8.1,3.9,1.000000,1.4,0.0
11684,2021-12-28,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,5.3,...,0.91,0.0,0.0,0.0,8.3,8.0,5.6,0.900000,1.3,0.0
11685,2021-12-29,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,7.9,...,4.94,0.0,0.0,0.0,8.5,7.8,6.9,2.300000,3.3,0.0
11686,2021-12-30,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,10.1,...,4.85,0.0,0.0,0.0,8.0,8.0,5.9,2.300000,3.2,0.0


In [180]:
# 주변국과 한국기후를  merge
cold_kr = pd.merge(df_coldwave,df_kr, how='right',on='일시')

In [181]:
cold_kr

,일시,재난일어난날,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,...,합계 일사,일 최심신적설,일 최심적설,합계 3시간 신적설,평균 전운량,평균 중하층운량,평균 지면온도,합계 대형증발량,합계 소형증발량,안개 계속 시간
0,1990-01-01,1990-01-01,서울,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,...,4.58,0.0,0.0,0.0,3.5,3.6,-1.5,2.721902,0.9,0.0
1,1990-01-02,1990-01-02,서울,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,...,3.44,0.1,0.1,0.1,3.5,3.8,-2.9,2.721902,0.8,0.0
2,1990-01-03,1990-01-03,서울,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,...,7.63,0.0,0.0,0.0,0.0,0.0,-4.4,2.721902,1.4,0.0
3,1990-01-04,1990-01-04,서울,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,...,5.84,0.0,0.0,0.0,0.5,1.8,-3.6,2.721902,1.0,0.0
4,1990-01-05,NaN,서울,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,...,0.36,0.6,0.6,0.6,9.0,3.6,-0.6,2.721902,1.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93499,2021-12-27,2021-12-27,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,...,1.37,0.5,1.2,0.5,8.1,8.1,3.9,1.000000,1.4,0.0
93500,2021-12-28,2021-12-28,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,...,0.91,0.0,0.0,0.0,8.3,8.0,5.6,0.900000,1.3,0.0
93501,2021-12-29,NaN,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,...,4.94,0.0,0.0,0.0,8.5,7.8,6.9,2.300000,3.3,0.0
93502,2021-12-30,NaN,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,...,4.85,0.0,0.0,0.0,8.0,8.0,5.9,2.300000,3.2,0.0


In [182]:
# cold_kr
# #19900101 -20211231
# df_near
# #19960101-20220831
cold_kr_near = pd.merge(cold_kr,df_near,how='left',on='일시')

In [183]:
cold_kr_near

,일시,재난일어난날,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,...,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
0,1990-01-01,1990-01-01,서울,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-01-02,1990-01-02,서울,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990-01-03,1990-01-03,서울,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-01-04,1990-01-04,서울,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990-01-05,NaN,서울,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2193499,2021-12-27,2021-12-27,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193500,2021-12-28,2021-12-28,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193501,2021-12-29,NaN,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193502,2021-12-30,NaN,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
# 재난일어난날 컬럼을 바이너리로 만들기 
cold_kr_near['재난일어난날'] = cold_kr_near['재난일어난날'].fillna(0)


In [185]:
cold_kr_near['재난일어난날'] = cold_kr_near['재난일어난날'].apply(lambda x: x if str(x) == '0' else '1')

In [186]:
cold_kr_near

,일시,재난일어난날,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,...,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
0,1990-01-01,1,서울,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-01-02,1,서울,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990-01-03,1,서울,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-01-04,1,서울,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990-01-05,0,서울,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2193499,2021-12-27,1,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193500,2021-12-28,1,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193501,2021-12-29,0,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193502,2021-12-30,0,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
cold_kr_near['재난일어난날']=cold_kr_near['재난일어난날'].astype(int)

In [188]:
cold_kr_near

,일시,재난일어난날,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,...,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
0,1990-01-01,1,서울,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-01-02,1,서울,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990-01-03,1,서울,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-01-04,1,서울,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990-01-05,0,서울,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2193499,2021-12-27,1,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193500,2021-12-28,1,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193501,2021-12-29,0,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193502,2021-12-30,0,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
cold_kr_near.drop('일시',axis=1,inplace=True)

In [190]:
cold_kr_near

,재난일어난날,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,...,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
0,1,서울,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,서울,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,4.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,서울,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,서울,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,서울,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,7.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2193499,1,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,6.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193500,1,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,5.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193501,0,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193502,0,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,10.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
cold_kr_near.drop('군집',axis=1,inplace=True)

In [192]:
cold_kr_near

,재난일어난날,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,...,도시명,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도
0,1,서울,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,서울,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,4.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,서울,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,서울,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,서울,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,7.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2193499,1,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,6.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193500,1,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,5.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193501,0,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2193502,0,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,10.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
cold_kr_near=pd.get_dummies(cold_kr_near)

In [ ]:
cold_kr_near

In [74]:
cold_kr_near.set_index('일시')

,재난일어난날,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,...,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
일시,,,,,,,,,,,,,,,,,,,,,
1996-01-01,1996-01-01,서울,-3.2,-7.9,2.5,0.0,0.0,6.0,340.0,3.5,...,-5.528571,-0.1,-9.9,53.571429,330.0,0.571429,0.0,1018.685714,-13.657143,n2
1996-01-01,1996-01-01,강릉,0.3,-4.7,6.0,0.0,0.0,10.4,250.0,5.0,...,-5.528571,-0.1,-9.9,53.571429,330.0,0.571429,0.0,1018.685714,-13.657143,n2
1996-01-01,1996-01-01,춘천,-7.9,-15.1,1.0,0.0,0.0,2.8,20.0,2.0,...,-5.528571,-0.1,-9.9,53.571429,330.0,0.571429,0.0,1018.685714,-13.657143,n2
1996-01-01,1996-01-01,청주,-3.6,-10.4,3.5,0.0,0.0,4.2,320.0,3.5,...,-5.528571,-0.1,-9.9,53.571429,330.0,0.571429,0.0,1018.685714,-13.657143,n2
1996-01-01,1996-01-01,전주,-2.4,-7.7,4.0,0.0,0.0,8.0,320.0,2.7,...,-5.528571,-0.1,-9.9,53.571429,330.0,0.571429,0.0,1018.685714,-13.657143,n2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,NaN,청주,22.9,20.6,26.2,1.9,2.2,8.1,50.0,4.9,...,29.100000,32.3,26.8,73.162500,290.0,4.875000,0.0,1013.795833,23.754167,n5
2021-08-31,NaN,전주,26.2,22.0,31.5,0.4,0.4,6.4,250.0,3.6,...,29.100000,32.3,26.8,73.162500,290.0,4.875000,0.0,1013.795833,23.754167,n5
2021-08-31,NaN,광주,26.5,22.9,31.0,0.0,0.0,7.6,230.0,4.7,...,29.100000,32.3,26.8,73.162500,290.0,4.875000,0.0,1013.795833,23.754167,n5


In [ ]:
cold_kr_near

In [88]:
cold_kr_near.tail(30)

,일시,재난일어난날,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,...,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
2174970,2021-08-28,NaN,춘천,23.2,19.3,28.5,0.0,0.0,4.7,200.0,...,28.641667,32.3,25.1,72.362500,40.0,4.958333,0.0,1014.325000,23.000000,n5
2174971,2021-08-28,NaN,청주,23.1,19.7,26.4,0.3,0.3,5.2,50.0,...,28.641667,32.3,25.1,72.362500,40.0,4.958333,0.0,1014.325000,23.000000,n5
2174972,2021-08-28,NaN,전주,23.6,20.3,29.0,0.2,0.2,6.0,340.0,...,28.641667,32.3,25.1,72.362500,40.0,4.958333,0.0,1014.325000,23.000000,n5
2174973,2021-08-28,NaN,광주,25.1,20.6,30.4,0.0,0.0,4.7,50.0,...,28.641667,32.3,25.1,72.362500,40.0,4.958333,0.0,1014.325000,23.000000,n5
2174974,2021-08-28,NaN,대구,25.6,20.9,31.9,0.0,0.0,5.0,290.0,...,28.641667,32.3,25.1,72.362500,40.0,4.958333,0.0,1014.325000,23.000000,n5
2174975,2021-08-28,NaN,제주,29.2,26.5,33.0,0.0,0.0,12.1,250.0,...,28.641667,32.3,25.1,72.362500,40.0,4.958333,0.0,1014.325000,23.000000,n5
2174976,2021-08-29,NaN,서울,23.7,21.9,26.3,0.7,0.9,5.6,340.0,...,28.908333,32.4,25.5,72.687500,290.0,5.041667,0.0,1014.754167,23.375000,n5
2174977,2021-08-29,NaN,강릉,25.4,21.8,30.6,0.0,0.0,7.3,200.0,...,28.908333,32.4,25.5,72.687500,290.0,5.041667,0.0,1014.754167,23.375000,n5
2174978,2021-08-29,NaN,춘천,22.8,20.2,26.8,1.1,1.2,3.6,340.0,...,28.908333,32.4,25.5,72.687500,290.0,5.041667,0.0,1014.754167,23.375000,n5
2174979,2021-08-29,NaN,청주,22.9,18.6,27.3,10.3,18.2,7.7,70.0,...,28.908333,32.4,25.5,72.687500,290.0,5.041667,0.0,1014.754167,23.375000,n5


In [22]:
cold_kr_near['재난일어난날'] = cold_kr_near['재난일어난날'].shift(periods=14,axis=0)

In [ ]:
cold_kr_near

In [ ]:
cold_kr_near.dropna(inplace=True)

In [ ]:
cold_kr_near

In [33]:
cold_kr_near['재난일어난날']=cold_kr_near['재난일어난날'].astype(int)
cold_kr_near

,재난일어난날,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,최대 풍속 풍향,...,도시명_원산,도시명_지난,도시명_창춘,도시명_청진,도시명_텐진,도시명_평양,도시명_함흥,도시명_해주,도시명_혜산,도시명_후쿠오카
14,1,1.7,-4.9,10.3,0.0,0.0,5.7,270.0,5.5,270.0,...,0,0,0,1,0,0,0,0,0,0
15,1,8.0,2.9,12.7,0.0,0.0,11.3,270.0,6.3,250.0,...,0,0,0,1,0,0,0,0,0,0
16,1,-4.5,-8.5,5.5,0.0,0.0,17.5,270.0,8.2,290.0,...,0,0,0,1,0,0,0,0,0,0
17,1,0.2,-6.8,8.8,0.0,0.0,24.8,250.0,13.7,250.0,...,0,0,0,1,0,0,0,0,0,0
18,1,-5.0,-12.1,2.5,0.0,0.0,15.6,250.0,10.0,250.0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2174995,1,22.9,20.6,26.2,1.9,2.2,8.1,50.0,4.9,20.0,...,0,0,0,0,0,0,0,0,0,1
2174996,1,26.2,22.0,31.5,0.4,0.4,6.4,250.0,3.6,270.0,...,0,0,0,0,0,0,0,0,0,1
2174997,1,26.5,22.9,31.0,0.0,0.0,7.6,230.0,4.7,230.0,...,0,0,0,0,0,0,0,0,0,1
2174998,1,24.4,21.5,29.1,13.6,15.2,7.0,140.0,5.0,140.0,...,0,0,0,0,0,0,0,0,0,1


In [37]:
y = cold_kr_near.재난일어난날
x = cold_kr_near.drop('재난일어난날',axis=1)

In [42]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.5,random_state=0,shuffle=True)

In [43]:
models = [lr,dtc,rfc,lgbm]
# result = []
for i in models:
    i.fit(x_train,y_train)
    pred = i.predict(x_test)
    proba = i.predict_proba(x_test)[:,1]
    
    print(i.__class__.__name__)
    print(classification_report(y_test,pred))
    print('roc_auc_score :',roc_auc_score(y_test,proba))
    print('-'*50)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1